In [2]:
import librosa
import numpy as np
import soundfile as sf
import os

def get_top_reference_windows(
    full_audio_file,
    sr_target=16000,
    window_sec=2.0,
    hop_sec=0.5,
    top_n=5,
    out_dir="reference_windows"
):
    os.makedirs(out_dir, exist_ok=True)

    y, sr = librosa.load(full_audio_file, sr=sr_target, mono=True)
    total_len = len(y)

    win_len = int(window_sec * sr)
    hop_len = int(hop_sec * sr)

    scored_windows = []

    idx = 0
    for start in range(0, total_len - win_len + 1, hop_len):
        end = start + win_len
        chunk = y[start:end]

        # score = loudness + harshness
        rms_val = librosa.feature.rms(y=chunk).mean()
        flat_val = librosa.feature.spectral_flatness(y=chunk).mean()
        score = rms_val + flat_val

        chunk_start_sec = start / sr
        chunk_end_sec = end / sr

        chunk_name = f"refcand_{idx:03d}_{chunk_start_sec:.2f}-{chunk_end_sec:.2f}.wav"
        chunk_path = os.path.join(out_dir, chunk_name)

        sf.write(chunk_path, chunk, sr)

        scored_windows.append({
            "chunk_path": chunk_path,
            "start_sec": chunk_start_sec,
            "end_sec": chunk_end_sec,
            "score": float(score)
        })
        idx += 1

    # sort by score desc (higher score = more intense/harsh)
    scored_windows = sorted(scored_windows, key=lambda x: x["score"], reverse=True)
    top_windows = scored_windows[:top_n]

    return top_windows


In [5]:
top_refs = get_top_reference_windows(
    full_audio_file="..\ReferenceAudio\BlueWhaleCrying.wav",  # your full clip
    window_sec=2.0,
    hop_sec=0.5,
    top_n=5,
    out_dir="reference_windows"
)
top_refs


<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\edupa\AppData\Local\Temp\ipykernel_20760\1149569542.py:2: SyntaxWarning: invalid escape sequence '\R'
  full_audio_file="..\ReferenceAudio\BlueWhaleCrying.wav",  # your full clip


[{'chunk_path': 'reference_windows\\refcand_034_17.00-19.00.wav',
  'start_sec': 17.0,
  'end_sec': 19.0,
  'score': 0.08015379309654236},
 {'chunk_path': 'reference_windows\\refcand_028_14.00-16.00.wav',
  'start_sec': 14.0,
  'end_sec': 16.0,
  'score': 0.05780639871954918},
 {'chunk_path': 'reference_windows\\refcand_029_14.50-16.50.wav',
  'start_sec': 14.5,
  'end_sec': 16.5,
  'score': 0.057611700147390366},
 {'chunk_path': 'reference_windows\\refcand_027_13.50-15.50.wav',
  'start_sec': 13.5,
  'end_sec': 15.5,
  'score': 0.054586637765169144},
 {'chunk_path': 'reference_windows\\refcand_017_8.50-10.50.wav',
  'start_sec': 8.5,
  'end_sec': 10.5,
  'score': 0.04877014830708504}]

Step 1 : the audio is 19 sec, we would like to chunk the audio to get reference windows for the whale screams then extract features

In [6]:
import numpy as np

FEATURE_KEYS = [
    "f0_mean",
    "f0_max",
    "f0_var",
    "f0_slope_std",
    "centroid_mean",
    "bandwidth_mean",
    "flatness_mean",
    "rms_max",
    "onsets_per_sec",
]

def extract_features_from_clip(file_path, sr_target=16000, fmin=200, fmax=8000):
    y, sr = librosa.load(file_path, sr=sr_target, mono=True)
    y, _ = librosa.effects.trim(y, top_db=40)

    duration_sec = len(y) / sr if len(y) > 0 else 0.0
    if duration_sec == 0:
        raise ValueError("Empty after trim")

    f0 = librosa.yin(y, fmin=fmin, fmax=fmax, sr=sr)
    f0 = f0[np.isfinite(f0)]
    f0 = f0[f0 > 0]

    if len(f0) == 0:
        f0_mean = 0.0
        f0_max = 0.0
        f0_var = 0.0
        f0_slope_std = 0.0
    else:
        f0_mean = float(np.mean(f0))
        f0_max = float(np.max(f0))
        f0_var = float(np.var(f0))
        f0_diff = np.diff(f0)
        f0_slope_std = float(np.std(f0_diff)) if len(f0_diff) > 0 else 0.0

    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    flatness = librosa.feature.spectral_flatness(y=y)
    rms = librosa.feature.rms(y=y)

    centroid_mean = float(np.mean(centroid))
    bandwidth_mean = float(np.mean(bandwidth))
    flatness_mean = float(np.mean(flatness))
    rms_max = float(np.max(rms))

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    onsets_per_sec = float(len(onset_frames) / duration_sec)

    feats_dict = {
        "f0_mean": f0_mean,
        "f0_max": f0_max,
        "f0_var": f0_var,
        "f0_slope_std": f0_slope_std,
        "centroid_mean": centroid_mean,
        "bandwidth_mean": bandwidth_mean,
        "flatness_mean": flatness_mean,
        "rms_max": rms_max,
        "onsets_per_sec": onsets_per_sec,
    }
    return feats_dict

def dict_to_vec(d):
    return np.array([d[k] for k in FEATURE_KEYS], dtype=float)

# build reference feature vectors
ref_vectors = []
for ref in top_refs:
    feats = extract_features_from_clip(ref["chunk_path"])
    ref_vec = dict_to_vec(feats)
    ref_vectors.append({
        "path": ref["chunk_path"],
        "start_sec": ref["start_sec"],
        "end_sec": ref["end_sec"],
        "vec": ref_vec
    })

len(ref_vectors)


5

Step 2 : Extract the top 5 window reference

In [ ]:
from scipy.spatial.distance import euclidean
import pandas as pd
import os

def slice_audio_to_windows(file_path, out_dir, window_sec=2.0, hop_sec=1.0, sr_target=16000):
    os.makedirs(out_dir, exist_ok=True)
    y, sr = librosa.load(file_path, sr=sr_target, mono=True)

    win_len = int(window_sec * sr)
    hop_len = int(hop_sec * sr)

    chunks_meta = []
    idx = 0
    for start in range(0, len(y) - win_len + 1, hop_len):
        end = start + win_len
        chunk = y[start:end]
        start_sec = start / sr
        end_sec = end / sr
        chunk_name = f"chunk_{idx:03d}_{start_sec:.2f}-{end_sec:.2f}.wav"
        chunk_path = os.path.join(out_dir, chunk_name)
        sf.write(chunk_path, chunk, sr)
        chunks_meta.append((chunk_path, start_sec, end_sec))
        idx += 1
    return chunks_meta

def score_chunks_against_reference_set(chunks_meta, ref_vectors):
    rows = []
    for (chunk_path, start_t, end_t) in chunks_meta:
        try:
            feats = extract_features_from_clip(chunk_path)
            cand_vec = dict_to_vec(feats)

            # distance to each reference vector
            dists = [euclidean(cand_vec, rv["vec"]) for rv in ref_vectors]
            best_dist = float(min(dists))

            rows.append({
                "chunk_path": chunk_path,
                "start_sec": start_t,
                "end_sec": end_t,
                "best_distance_to_refset": best_dist,
                **feats
            })
        except Exception as e:
            print(f"Skipping {chunk_path}: {e}")

    df = pd.DataFrame(rows)
    df = df.sort_values("best_distance_to_refset", ascending=True).reset_index(drop=True)
    return df

# Example usage on your same 19s clip for now:
chunks_meta_full = slice_audio_to_windows(
    file_path="..\ReferenceAudio\BlueWhaleCrying.wav",
    out_dir="chunks_fullscan",
    window_sec=2.0,
    hop_sec=1.0,
    sr_target=16000
)

df_ranked = score_chunks_against_reference_set(chunks_meta_full, ref_vectors)
df_ranked.head(10)


<>:53: SyntaxWarning: invalid escape sequence '\R'
<>:53: SyntaxWarning: invalid escape sequence '\R'
C:\Users\edupa\AppData\Local\Temp\ipykernel_20760\1356485577.py:53: SyntaxWarning: invalid escape sequence '\R'
  file_path="..\ReferenceAudio\BlueWhaleCrying.wav",


,chunk_path,start_sec,end_sec,best_distance_to_refset,f0_mean,f0_max,f0_var,f0_slope_std,centroid_mean,bandwidth_mean,flatness_mean,rms_max,onsets_per_sec
0,chunks_fullscan\chunk_014_14.00-16.00.wav,14.0,16.0,0.000000,776.369638,8000.000000,1.802676e+06,1320.491553,1212.751003,1404.585009,0.003056,0.152773,8.500000
1,chunks_fullscan\chunk_017_17.00-19.00.wav,17.0,19.0,0.000000,771.000624,1196.962157,1.116645e+05,222.893414,1486.291541,1545.965508,0.009390,0.092784,7.291667
2,chunks_fullscan\chunk_016_16.00-18.00.wav,16.0,18.0,10628.026113,704.500332,1203.884522,1.010384e+05,199.399892,1305.616797,1482.750050,0.003145,0.093351,5.000000
3,chunks_fullscan\chunk_008_8.00-10.00.wav,8.0,10.0,13746.939339,1058.087187,8000.000000,1.816421e+06,1334.790406,1210.058464,1414.651708,0.002724,0.084551,4.000000
4,chunks_fullscan\chunk_015_15.00-17.00.wav,15.0,17.0,17492.911404,637.897603,997.287587,8.507758e+04,139.486185,1273.259913,1458.914905,0.003155,0.106051,5.000000
5,chunks_fullscan\chunk_009_9.00-11.00.wav,9.0,11.0,35466.903067,853.507192,935.326228,3.212119e+04,102.614405,1249.021352,1428.509372,0.002551,0.066511,5.500000
6,chunks_fullscan\chunk_012_12.00-14.00.wav,12.0,14.0,61927.002623,971.104669,8000.000000,1.711132e+06,1382.642279,1246.867881,1467.818420,0.003165,0.076992,4.000000
7,chunks_fullscan\chunk_002_2.00-4.00.wav,2.0,4.0,425363.142602,1377.507906,8000.000000,3.810958e+06,907.389438,1295.008432,1514.106850,0.003222,0.057046,6.000000
8,chunks_fullscan\chunk_010_10.00-12.00.wav,10.0,12.0,504637.684655,1420.221650,8000.000000,3.890233e+06,1287.280314,1195.204807,1477.037241,0.002887,0.085110,5.500000
9,chunks_fullscan\chunk_013_13.00-15.00.wav,13.0,15.0,756046.126994,1174.971782,8000.000000,4.141641e+06,1908.128125,1064.049578,1393.843052,0.002571,0.155056,5.000000


Step 3 : Load the dataset and compare based oon the references to get the similar whale calls to the audio

In [43]:
import os
import io
import numpy as np
import soundfile as sf
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

# 1. Load the Arrow table directly (no to_pandas yet)
parquet_path = r"..\humpbacks-orcasound-em-hW-data\data\train-00000-of-00006.parquet"
table = pq.read_table(parquet_path)

# 2. Let's inspect schema so we know column names and types
print("Schema:")
print(table.schema)

# 3. Convert each column we care about to Python lists (except audio)
# We'll handle 'audio' manually
cols = table.column_names
print("Columns:", cols)


Schema:
Selection: int64
BeginTime: double
EndTime: double
LowFreq: double
HighFreq: double
CallType: string
Filename: string
VocalizationDuration: double
Audio: struct<bytes: binary, path: string>
  child 0, bytes: binary
  child 1, path: string
-- schema metadata --
huggingface: '{"info": {"features": {"Selection": {"dtype": "int64", "_ty' + 442
Columns: ['Selection', 'BeginTime', 'EndTime', 'LowFreq', 'HighFreq', 'CallType', 'Filename', 'VocalizationDuration', 'Audio']


In [ ]:
import os
import io
import soundfile as sf
import pandas as pd

os.makedirs("public_raw_300", exist_ok=True)

max_clips = 300
export_meta = []

# Get column indices to avoid string lookup in loop
col_idx = {name: i for i, name in enumerate(table.column_names)}

# Helper to get a value from a row safely
def get_value(col_name, row_idx):
    if col_name not in col_idx:
        return None
    col = table.column(col_idx[col_name])
    # Arrow -> Python scalar
    return col[row_idx].as_py()

# We'll loop over row indexes directly
num_rows = table.num_rows
count = 0

for row_idx in range(num_rows):
    if count >= max_clips:
        break

    # 1. Get the audio struct cell (Arrow struct: {'bytes': ..., 'path': ...})
    # Try "audio" first, then "Audio"
    audio_cell = None
    if "audio" in col_idx:
        audio_cell = table.column(col_idx["audio"])[row_idx].as_py()
    elif "Audio" in col_idx:
        audio_cell = table.column(col_idx["Audio"])[row_idx].as_py()
    else:
        print("No audio column found, stopping.")
        break

    # audio_cell should now be a dict like {'bytes': b'RIFF...', 'path': '...'}
    if audio_cell is None or "bytes" not in audio_cell:
        # No usable audio => skip
        continue

    audio_bytes = audio_cell["bytes"]

    # 2. Decode bytes into waveform using soundfile
    try:
        with sf.SoundFile(io.BytesIO(audio_bytes)) as f:
            y = f.read(dtype="float32")
            sr = f.samplerate
    except RuntimeError as e:
        # sometimes the bytes might not be WAV; skip if decode fails
        print(f"Skipping row {row_idx}: could not decode audio ({e})")
        continue

    # 3. Save to disk
    out_name = f"humpback_{count:04d}.wav"
    out_path = os.path.join("public_raw_300", out_name)
    sf.write(out_path, y, sr)

    # 4. Collect metadata from other columns
    export_meta.append({
        "filename": out_path,
        "sr": sr,
        "Selection": get_value("Selection", row_idx),
        "BeginTime": get_value("BeginTime", row_idx),
        "EndTime": get_value("EndTime", row_idx),
        "LowFreq": get_value("LowFreq", row_idx),
        "HighFreq": get_value("HighFreq", row_idx),
        "CallType": get_value("CallType", row_idx),
        "VocalizationDuration": get_value("VocalizationDuration", row_idx),
        "OriginalFile": get_value("Filename", row_idx),
        "SourceParquet": os.path.basename(parquet_path),
    })

    count += 1

print(f"✅ Exported {count} clips from {parquet_path}")

# 5. Save metadata for these clips
meta_df = pd.DataFrame(export_meta)
meta_df.to_csv("public_raw_300/metadata_raw.csv", index=False)



✅ Exported 253 clips from ..\humpbacks-orcasound-em-hW-data\data\train-00000-of-00006.parquet
                           filename     sr  Selection  BeginTime   EndTime  \
0  public_raw_300\humpback_0000.wav  44100          1   1.812867  3.187133   
1  public_raw_300\humpback_0001.wav  44100          2   0.618912  4.381088   
2  public_raw_300\humpback_0002.wav  44100          3   2.266082  2.733918   
3  public_raw_300\humpback_0003.wav  44100          4   1.125734  3.874266   
4  public_raw_300\humpback_0004.wav  44100          5   2.071151  2.928849   

    LowFreq  HighFreq         CallType  VocalizationDuration  \
0   263.518  2845.995  Descending_moan              1.374266   
1   263.518  1791.923  Descending_moan              3.762175   
2  2898.699  3531.142          Whistle              0.467835   
3   158.111  1844.626  Descending_moan              2.748532   
4  1106.776  3056.809          Whistle              0.857698   

                                 OriginalFile       

In [45]:
meta_df.head()

,filename,sr,Selection,BeginTime,EndTime,LowFreq,HighFreq,CallType,VocalizationDuration,OriginalFile,SourceParquet
0,public_raw_300\humpback_0000.wav,44100,1,1.812867,3.187133,263.518,2845.995,Descending_moan,1.374266,211026-133018-OS-humpback-47min-clip_1.wav,train-00000-of-00006.parquet
1,public_raw_300\humpback_0001.wav,44100,2,0.618912,4.381088,263.518,1791.923,Descending_moan,3.762175,211026-133018-OS-humpback-47min-clip_2.wav,train-00000-of-00006.parquet
2,public_raw_300\humpback_0002.wav,44100,3,2.266082,2.733918,2898.699,3531.142,Whistle,0.467835,211026-133018-OS-humpback-47min-clip_3.wav,train-00000-of-00006.parquet
3,public_raw_300\humpback_0003.wav,44100,4,1.125734,3.874266,158.111,1844.626,Descending_moan,2.748532,211026-133018-OS-humpback-47min-clip_4.wav,train-00000-of-00006.parquet
4,public_raw_300\humpback_0004.wav,44100,5,2.071151,2.928849,1106.776,3056.809,Whistle,0.857698,211026-133018-OS-humpback-47min-clip_5.wav,train-00000-of-00006.parquet


In [58]:
import numpy as np
import scipy.signal as signal
import librosa

def safe_filter(y, sr, low_hz=200, high_hz=8000, order=5):
    nyq = 0.5 * sr

    # clamp high cutoff to below Nyquist
    effective_high = min(high_hz, nyq * 0.9)
    effective_low = max(low_hz, 1.0)  # avoid 0 Hz

    # if low is now >= high, fall back to high-pass only
    if effective_low >= effective_high:
        # design high-pass at effective_low
        Wn = effective_low / nyq
        if Wn >= 1:
            # can't filter at all, just return original
            return y
        b, a = signal.butter(order, Wn, btype='highpass')
        return signal.lfilter(b, a, y)

    # normal band-pass
    low_norm = effective_low / nyq
    high_norm = effective_high / nyq

    # if high_norm >=1, invalid -> clamp
    if high_norm >= 1:
        high_norm = 0.99
    if low_norm <= 0:
        low_norm = 0.0001
    if low_norm >= high_norm:
        # weird edge, just return original
        return y

    b, a = signal.butter(order, [low_norm, high_norm], btype='bandpass')
    y_filt = signal.lfilter(b, a, y)
    return y_filt


Step 4 : Extract Features , build top k vectors and compute similarity to extract the nearest audios to the reference (On going)

In [47]:
import numpy as np
import librosa
import librosa.feature
import scipy.signal as signal

def bandpass_filter(y, sr, low_hz=200, high_hz=8000, order=5):
    nyq = 0.5 * sr
    low = low_hz / nyq
    high = high_hz / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    y_filt = signal.lfilter(b, a, y)
    return y_filt

def extract_features_from_clip_denoised(
    file_path,
    sr_target=16000,
    fmin=200,
    fmax=8000,
    activity_db_above_median=6,
    hop_length=512
):
    # 1. load mono audio
    y, sr = librosa.load(file_path, sr=sr_target, mono=True)

    # 2. trim leading/trailing silence-ish
    y, _ = librosa.effects.trim(y, top_db=40)

    if len(y) == 0:
        raise ValueError("Clip is silent after trim")

    duration_sec = len(y) / sr

    # 3. band-pass filter to reduce ocean rumble / very low freq flow noise
    y_bp = bandpass_filter(y, sr, low_hz=fmin, high_hz=fmax)

    # 4. frame-wise RMS to find "active" parts
    frame_rms = librosa.feature.rms(y=y_bp, frame_length=2048, hop_length=hop_length)[0]
    median_rms = np.median(frame_rms)
    # mark frames that are clearly above baseline
    active_mask = frame_rms > (median_rms * (10**(activity_db_above_median/20.0)))

    # frame times to later compute timing features
    frame_times = librosa.frames_to_time(
        np.arange(len(frame_rms)),
        sr=sr,
        hop_length=hop_length
    )

    # Helper: keep only active portions for pitch + spectral stats
    def active_frames_stats(feature_array):
        # feature_array shape = (feature_dim, n_frames) or (n_frames,)
        # we subset columns/frames where active_mask is True
        if feature_array.ndim == 2:
            active_vals = feature_array[:, active_mask]
        else:
            active_vals = feature_array[active_mask]
        if active_vals.size == 0:
            return {
                "mean": float(np.mean(feature_array)),
                "max": float(np.max(feature_array)),
                "var": float(np.var(feature_array)),
                "std": float(np.std(feature_array)),
            }
        return {
            "mean": float(np.mean(active_vals)),
            "max": float(np.max(active_vals)),
            "var": float(np.var(active_vals)),
            "std": float(np.std(active_vals)),
        }

    # 5. pitch curve (fundamental frequency)
    f0 = librosa.yin(y_bp, fmin=fmin, fmax=fmax, sr=sr, frame_length=2048, hop_length=hop_length)
    # clean pitch: valid only if finite and >0
    f0[~np.isfinite(f0)] = np.nan
    f0[f0 <= 0] = np.nan

    # pitch derivative (how unstable)
    f0_diff = np.diff(f0)
    f0_diff[~np.isfinite(f0_diff)] = np.nan

    # 6. spectral features on band-passed signal
    centroid = librosa.feature.spectral_centroid(y=y_bp, sr=sr, hop_length=hop_length)[0]
    bandwidth = librosa.feature.spectral_bandwidth(y=y_bp, sr=sr, hop_length=hop_length)[0]
    flatness = librosa.feature.spectral_flatness(y=y_bp, hop_length=hop_length)[0]

    # 7. intensity features (RMS burstiness)
    rms_full = frame_rms  # on bandpassed signal
    rms_stats = {
        "rms_mean": float(np.mean(rms_full)),
        "rms_max": float(np.max(rms_full)),
        "rms_var": float(np.var(rms_full)),
    }

    # 8. Activity rate (how many bursts / urgency)
    onset_env = librosa.onset.onset_strength(y=y_bp, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    onsets_per_sec = float(len(onset_frames) / duration_sec)

    # 9. Active-frame stats for biologically interesting stuff:
    pitch_stats = {
        "f0_mean": float(np.nanmean(f0)),
        "f0_max": float(np.nanmax(f0)),
        "f0_var": float(np.nanvar(f0)),
        "f0_slope_std": float(np.nanstd(f0_diff)),
    }

    centroid_stats = active_frames_stats(centroid)
    bandwidth_stats = active_frames_stats(bandwidth)
    flatness_stats = active_frames_stats(flatness)

    feats = {
        # duration/context
        "duration_sec": duration_sec,

        # pitch / instability
        "f0_mean": pitch_stats["f0_mean"],
        "f0_max": pitch_stats["f0_max"],
        "f0_var": pitch_stats["f0_var"],
        "f0_slope_std": pitch_stats["f0_slope_std"],

        # spectral shape (where and how wide the energy is)
        "centroid_mean": centroid_stats["mean"],
        "centroid_max": centroid_stats["max"],
        "bandwidth_mean": bandwidth_stats["mean"],
        "bandwidth_max": bandwidth_stats["max"],

        # harshness / noisiness
        "flatness_mean": flatness_stats["mean"],
        "flatness_max": flatness_stats["max"],

        # intensity / urgency
        "rms_mean": rms_stats["rms_mean"],
        "rms_max": rms_stats["rms_max"],
        "rms_var": rms_stats["rms_var"],
        "onsets_per_sec": onsets_per_sec,
    }

    return feats


In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import scipy.signal as signal
from scipy.spatial.distance import euclidean
from tqdm import tqdm

def extract_features_from_clip_denoised(
    file_path,
    sr_target=16000,
    fmin=200,
    fmax=8000,
    activity_db_above_median=6,
    hop_length=512
):
    # 1. load mono audio
    y, sr = librosa.load(file_path, sr=None, mono=True)  # keep native sr first

    # if sr is super low (< 2kHz) librosa.yin etc may break. We will upsample.
    if sr < sr_target:
        # resample to sr_target for consistency
        y = librosa.resample(y, orig_sr=sr, target_sr=sr_target)
        sr = sr_target
    else:
        # if higher than target, downsample to sr_target
        y = librosa.resample(y, orig_sr=sr, target_sr=sr_target)
        sr = sr_target

    # basic duration guard
    duration_sec = len(y) / sr
    if duration_sec < 0.3:
        raise ValueError("Clip too short to analyze (<0.3s)")

    # 2. trim leading/trailing silence-ish
    y_trim, _ = librosa.effects.trim(y, top_db=40)
    if len(y_trim) > 0:
        y = y_trim

    # update duration after trim
    duration_sec = len(y) / sr
    if duration_sec < 0.3:
        raise ValueError("Clip effectively empty after trim")

    # 3. robust filter to remove low-frequency ocean rumble, etc.
    y_bp = safe_filter(y, sr, low_hz=fmin, high_hz=fmax)

    # 4. frame-wise RMS (energy curve)
    frame_rms = librosa.feature.rms(y=y_bp, frame_length=2048, hop_length=hop_length)[0]
    median_rms = np.median(frame_rms) if np.median(frame_rms) > 0 else (np.mean(frame_rms) + 1e-9)
    thresh = median_rms * (10**(activity_db_above_median/20.0))
    active_mask = frame_rms > thresh

    # pitch track
    try:
        f0 = librosa.yin(
            y_bp,
            fmin=fmin,
            fmax=fmax,
            sr=sr,
            frame_length=2048,
            hop_length=hop_length
        )
    except Exception:
        # fallback: no pitch detectable
        f0 = np.full(1, np.nan)

    f0[~np.isfinite(f0)] = np.nan
    f0[f0 <= 0] = np.nan
    f0_diff = np.diff(f0)
    f0_diff[~np.isfinite(f0_diff)] = np.nan

    # spectral features
    centroid = librosa.feature.spectral_centroid(y=y_bp, sr=sr, hop_length=hop_length)[0]
    bandwidth = librosa.feature.spectral_bandwidth(y=y_bp, sr=sr, hop_length=hop_length)[0]
    flatness = librosa.feature.spectral_flatness(y=y_bp, hop_length=hop_length)[0]

    def active_stats(arr):
        # arr: shape (n_frames,) or (n_features,n_frames)
        if arr.ndim > 1:
            arr_use = arr[:, active_mask] if active_mask.shape[0] == arr.shape[1] else arr
        else:
            if active_mask.shape[0] == arr.shape[0]:
                arr_use = arr[active_mask]
            else:
                arr_use = arr

        if arr_use.size == 0:
            arr_use = arr

        return {
            "mean": float(np.nanmean(arr_use)),
            "max":  float(np.nanmax(arr_use)),
            "var":  float(np.nanvar(arr_use)),
            "std":  float(np.nanstd(arr_use)),
        }

    centroid_stats = active_stats(centroid)
    bandwidth_stats = active_stats(bandwidth)
    flatness_stats = active_stats(flatness)

    rms_mean = float(np.mean(frame_rms))
    rms_max  = float(np.max(frame_rms))
    rms_var  = float(np.var(frame_rms))

    onset_env = librosa.onset.onset_strength(y=y_bp, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    onsets_per_sec = float(len(onset_frames) / duration_sec)

    feats = {
        "duration_sec": duration_sec,
        "f0_mean": float(np.nanmean(f0)),
        "f0_max": float(np.nanmax(f0)),
        "f0_var": float(np.nanvar(f0)),
        "f0_slope_std": float(np.nanstd(f0_diff)),
        "centroid_mean": centroid_stats["mean"],
        "bandwidth_mean": bandwidth_stats["mean"],
        "flatness_mean": flatness_stats["mean"],
        "rms_mean": rms_mean,
        "rms_max": rms_max,
        "rms_var": rms_var,
        "onsets_per_sec": onsets_per_sec,
    }

    return feats


In [56]:
import numpy as np

# How many top scream-like windows from your whale clip to treat as "panic prototypes"
TOP_REF_WINDOWS = 5  # you can adjust 3, 5, 8...

# take the top N most panic-like chunks
panic_proto_df = df_ranked.sort_values("best_distance_to_refset", ascending=True).head(TOP_REF_WINDOWS)

# list of feature keys we care about (must match downstream)
FEATURE_KEYS = [
    "f0_mean",
    "f0_max",
    "f0_var",
    "f0_slope_std",
    "centroid_mean",
    "bandwidth_mean",
    "flatness_mean",
    "rms_max",
    "onsets_per_sec",
]

def feats_to_vec_from_row(row):
    return np.array([row[k] for k in FEATURE_KEYS], dtype=float)

ref_feature_vectors = []
for _, r in panic_proto_df.iterrows():
    ref_feature_vectors.append(feats_to_vec_from_row(r))

print(f"Built {len(ref_feature_vectors)} panic reference vectors from whale clip.")


Built 5 panic reference vectors from whale clip.


step 5 : new datset Features from the datset

In [60]:
import os
from tqdm import tqdm
import pandas as pd

public_folder = "public_raw_300"

public_feature_rows = []
num_tried = 0
num_ok = 0
num_err = 0

for wav_name in tqdm(os.listdir(public_folder)):
    if not wav_name.lower().endswith(".wav"):
        continue

    full_path = os.path.join(public_folder, wav_name)
    num_tried += 1

    try:
        feats = extract_features_from_clip_denoised(full_path)

        row = {
            "file": full_path,
            "f0_mean": feats["f0_mean"],
            "f0_max": feats["f0_max"],
            "f0_var": feats["f0_var"],
            "f0_slope_std": feats["f0_slope_std"],
            "centroid_mean": feats["centroid_mean"],
            "bandwidth_mean": feats["bandwidth_mean"],
            "flatness_mean": feats["flatness_mean"],
            "rms_max": feats["rms_max"],
            "onsets_per_sec": feats["onsets_per_sec"],
            "duration_sec": feats["duration_sec"],
            "rms_mean": feats["rms_mean"],
            "rms_var": feats["rms_var"],
        }

        public_feature_rows.append(row)
        num_ok += 1

    except Exception as e:
        print(f"[SKIP] {full_path} -> {e}")
        num_err += 1

public_df = pd.DataFrame(public_feature_rows)
print("Tried:", num_tried, "| OK:", num_ok, "| Err:", num_err)
print("public_df shape:", public_df.shape)
public_df.head()


100%|██████████| 254/254 [00:10<00:00, 23.47it/s]

Tried: 253 | OK: 253 | Err: 0
public_df shape: (253, 13)


,file,f0_mean,f0_max,f0_var,f0_slope_std,centroid_mean,bandwidth_mean,flatness_mean,rms_max,onsets_per_sec,duration_sec,rms_mean,rms_var
0,public_raw_300\humpback_0000.wav,385.905202,879.121603,21376.957764,130.838933,2599.256745,1927.750690,0.095066,0.005341,3.2,5.0,0.003397,2.088397e-07
1,public_raw_300\humpback_0001.wav,482.668895,1181.978083,57401.005484,169.293557,2610.603717,1849.050314,0.072034,0.008093,3.8,5.0,0.004156,8.390921e-07
2,public_raw_300\humpback_0002.wav,558.235715,3229.183047,356393.865003,367.718382,2650.093017,1974.882135,0.091636,0.015264,1.4,5.0,0.003648,3.968836e-06
3,public_raw_300\humpback_0003.wav,378.060943,897.520422,12545.149331,124.531777,2371.970462,1881.881597,0.070464,0.016047,1.6,5.0,0.004193,7.896844e-06
4,public_raw_300\humpback_0004.wav,398.989217,1585.284994,34052.004791,187.573653,2783.512659,1868.826415,0.037440,0.008155,4.2,5.0,0.003412,7.969308e-07
